# Classifier Evaluation Lab

* Copy&paste your model for homework5 model
* Add grid search and train
* Compare performance
* Which one is better? Explain?

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

df = pd.read_csv('https://raw.githubusercontent.com/msaricaumbc/DS_data/master/ds602/log_reg/employee-turnover-balanced.csv')
df.head()

,left_company,age,frequency_of_travel,department,commuting_distance,education,satisfaction_with_environment,gender,seniority_level,position,satisfaction_with_job,married_or_single,last_raise_pct,last_performance_rating,total_years_working,years_at_company,years_in_current_job,years_since_last_promotion,years_with_current_supervisor
0,No,37,Travel_Rarely,Sales,16,4,4,Male,2,Sales Executive,3,Divorced,19,3,9,1,0,0,0
1,No,39,Travel_Rarely,Research & Development,3,2,3,Male,2,Laboratory Technician,3,Divorced,15,3,11,10,8,0,7
2,No,52,Travel_Frequently,Research & Development,25,4,3,Female,4,Manufacturing Director,4,Married,22,4,31,9,8,0,0
3,No,50,Non-Travel,Sales,1,3,4,Female,2,Sales Executive,3,Married,12,3,19,18,7,0,13
4,No,44,Travel_Rarely,Research & Development,4,3,4,Male,2,Healthcare Representative,2,Single,12,3,10,5,2,2,3


In [2]:
numerical_variables = ['age','commuting_distance','last_raise_pct','total_years_working','years_at_company','years_in_current_job','years_since_last_promotion','years_with_current_supervisor']
numerical_variables

['age',
 'commuting_distance',
 'last_raise_pct',
 'total_years_working',
 'years_at_company',
 'years_in_current_job',
 'years_since_last_promotion',
 'years_with_current_supervisor']

In [3]:
categorical_variables = [cat for cat in df.columns if cat not in numerical_variables]
categorical_variables

['left_company',
 'frequency_of_travel',
 'department',
 'education',
 'satisfaction_with_environment',
 'gender',
 'seniority_level',
 'position',
 'satisfaction_with_job',
 'married_or_single',
 'last_performance_rating']

In [4]:
categorical_variables.remove('left_company')

In [5]:
categorical_variables

['frequency_of_travel',
 'department',
 'education',
 'satisfaction_with_environment',
 'gender',
 'seniority_level',
 'position',
 'satisfaction_with_job',
 'married_or_single',
 'last_performance_rating']

In [6]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

def var_in_fac(X):
    # Calculating VIF
    var_in_fac = pd.DataFrame()
    var_in_fac["Variables"] = df[numerical_variables].columns
    var_in_fac["Variance Inflation Factor"] = [variance_inflation_factor(df[numerical_variables].values, i) for i in range(df[numerical_variables].shape[1])]

    return(var_in_fac)

In [7]:
X = df.iloc[:,:-1]
var_in_fac(X)

,Variables,Variance Inflation Factor
0,age,16.669850
1,commuting_distance,2.389730
2,last_raise_pct,10.656507
3,total_years_working,7.622769
4,years_at_company,9.588173
5,years_in_current_job,7.656341
6,years_since_last_promotion,2.590716
7,years_with_current_supervisor,6.888652


In [8]:
df = df.drop(['age','years_at_company','years_in_current_job'],axis=1)

In [9]:
numerical_variables = ['commuting_distance','last_raise_pct','total_years_working','years_since_last_promotion','years_with_current_supervisor']
numerical_variables

['commuting_distance',
 'last_raise_pct',
 'total_years_working',
 'years_since_last_promotion',
 'years_with_current_supervisor']

In [10]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

def calc_vif(X):

    # Calculating VIF
    vif = pd.DataFrame()
    vif["variables"] = df[numerical_variables].columns
    vif["VIF"] = [variance_inflation_factor(df[numerical_variables].values, i) for i in range(df[numerical_variables].shape[1])]

    return(vif)

In [11]:
X = df.iloc[:,:-1]
calc_vif(X)

,variables,VIF
0,commuting_distance,2.365671
1,last_raise_pct,3.849253
2,total_years_working,3.598308
3,years_since_last_promotion,2.334495
4,years_with_current_supervisor,3.842648


In [12]:
y = df['left_company'] # Target column is left_company
X = df[[x for x in df.columns if x != 'left_company']] # Columns except the target column

In [13]:
X.columns

Index(['frequency_of_travel', 'department', 'commuting_distance', 'education',
       'satisfaction_with_environment', 'gender', 'seniority_level',
       'position', 'satisfaction_with_job', 'married_or_single',
       'last_raise_pct', 'last_performance_rating', 'total_years_working',
       'years_since_last_promotion', 'years_with_current_supervisor'],
      dtype='object')

In [14]:
# insert code here
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)

print(f'Training samples: {X_train.shape[0]:,}')
print(f'Test samples: {X_test.shape[0]:,}')

Training samples: 800
Test samples: 200


In [15]:
from sklearn.pipeline import Pipeline # import pipeline
from sklearn.compose import ColumnTransformer # import column transformer for transforming the data.
from sklearn.preprocessing import OneHotEncoder # used for representing categorical columns as numeric.
from sklearn.preprocessing import StandardScaler # used for standardizing the values

In [16]:
# insert code here
num_pipeline = Pipeline([('standardize_num', StandardScaler())])
num_data_processing_pipeline = ColumnTransformer(transformers=[('proc_numeric', num_pipeline, numerical_variables)])
print(num_data_processing_pipeline)

ColumnTransformer(transformers=[('proc_numeric',
                                 Pipeline(steps=[('standardize_num',
                                                  StandardScaler())]),
                                 ['commuting_distance', 'last_raise_pct',
                                  'total_years_working',
                                  'years_since_last_promotion',
                                  'years_with_current_supervisor'])])


In [17]:
num_transformed_data = num_data_processing_pipeline.fit_transform(X_train)
num_transformed_data

array([[-0.67905134,  2.58364868,  1.70122817, -0.0140457 , -0.49140969],
       [ 2.43872342,  1.78745494,  0.51455477,  0.3164413 ,  1.25582476],
       [-0.80376234,  0.19506747,  0.64640737, -0.67501968, -0.78261543],
       ...,
       [-0.80376234, -1.1319221 , -0.01285563,  2.29936325,  0.96461902],
       [-0.92847333, -0.07033045,  3.01975417,  1.30790227,  1.54703051],
       [ 0.19392559,  0.46046538, -0.01285563, -0.67501968,  1.54703051]])

In [18]:
num_transformed_data.shape

(800, 5)

In [19]:
cat_pipeline = Pipeline([('create_dummies_cats', OneHotEncoder(handle_unknown='ignore', drop='first'))])
cat_processing_pipeline = ColumnTransformer(transformers=[('create_dummies', cat_pipeline, categorical_variables)])

print(cat_processing_pipeline)

ColumnTransformer(transformers=[('create_dummies',
                                 Pipeline(steps=[('create_dummies_cats',
                                                  OneHotEncoder(drop='first',
                                                                handle_unknown='ignore'))]),
                                 ['frequency_of_travel', 'department',
                                  'education', 'satisfaction_with_environment',
                                  'gender', 'seniority_level', 'position',
                                  'satisfaction_with_job', 'married_or_single',
                                  'last_performance_rating'])])


In [20]:
cat_transformed_data = cat_processing_pipeline.fit_transform(X_train)
cat_transformed_data

<800x30 sparse matrix of type '<class 'numpy.float64'>'
	with 5843 stored elements in Compressed Sparse Row format>

In [21]:
cat_transformed_data.shape

(800, 30)

In [22]:
# insert code here

processing_pipeline = ColumnTransformer(transformers=[('proc_numeric', num_pipeline, numerical_variables),
                                                      ('create_dummies', cat_pipeline, categorical_variables)])

print(processing_pipeline)

ColumnTransformer(transformers=[('proc_numeric',
                                 Pipeline(steps=[('standardize_num',
                                                  StandardScaler())]),
                                 ['commuting_distance', 'last_raise_pct',
                                  'total_years_working',
                                  'years_since_last_promotion',
                                  'years_with_current_supervisor']),
                                ('create_dummies',
                                 Pipeline(steps=[('create_dummies_cats',
                                                  OneHotEncoder(drop='first',
                                                                handle_unknown='ignore'))]),
                                 ['frequency_of_travel', 'department',
                                  'education', 'satisfaction_with_environment',
                                  'gender', 'seniority_level', 'position',
                          

In [23]:
from sklearn.linear_model import LogisticRegression
from collections import defaultdict
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import roc_auc_score

In [24]:
from sklearn.linear_model import LinearRegression

modeling_pipeline = Pipeline([
        ('data_processing', processing_pipeline),
        ('logreg', LogisticRegression(penalty=None))]
    )
modeling_pipeline.fit(X_train, y_train) # fit the training dataset for pipeline model

C:\Users\CHANDRIKA\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Pipeline(steps=[('data_processing',
                 ColumnTransformer(transformers=[('proc_numeric',
                                                  Pipeline(steps=[('standardize_num',
                                                                   StandardScaler())]),
                                                  ['commuting_distance',
                                                   'last_raise_pct',
                                                   'total_years_working',
                                                   'years_since_last_promotion',
                                                   'years_with_current_supervisor']),
                                                 ('create_dummies',
                                                  Pipeline(steps=[('create_dummies_cats',
                                                                   OneHotEncoder(drop='first',
                                                                                 handle_unknown='ignore'))]),
                                                  ['frequency_of_travel',
                                                   'department', 'education',
                                                   'satisfaction_with_environment',
                                                   'gender', 'seniority_level',
                                                   'position',
                                                   'satisfaction_with_job',
                                                   'married_or_single',
                                                   'last_performance_rating'])])),
                ('logreg', LogisticRegression(penalty=None))])

In [25]:
y_train_predictions = modeling_pipeline.predict_proba(X_train)
y_train_predictions[:5]

array([[0.95854603, 0.04145397],
       [0.28781184, 0.71218816],
       [0.22227229, 0.77772771],
       [0.47543365, 0.52456635],
       [0.95550294, 0.04449706]])

In [26]:
from sklearn.metrics import roc_auc_score

roc_auc_score(y_train, y_train_predictions[:,1])

0.8305154664949904

In [27]:
from sklearn.metrics import confusion_matrix

y_train_predictions = modeling_pipeline.predict(X_train)

print(confusion_matrix(y_train, y_train_predictions))

[[287 110]
 [ 92 311]]


In [28]:
from sklearn.metrics import classification_report
print(classification_report(y_train, y_train_predictions))

              precision    recall  f1-score   support

          No       0.76      0.72      0.74       397
         Yes       0.74      0.77      0.75       403

    accuracy                           0.75       800
   macro avg       0.75      0.75      0.75       800
weighted avg       0.75      0.75      0.75       800



In [29]:
y_test_predictions = modeling_pipeline.predict_proba(X_test)
y_test_predictions[:5]

array([[0.80429368, 0.19570632],
       [0.05456463, 0.94543537],
       [0.21991674, 0.78008326],
       [0.90850457, 0.09149543],
       [0.25865473, 0.74134527]])

In [30]:
from sklearn.metrics import roc_auc_score

roc_auc_score(y_test, y_test_predictions[:,1])

0.7533780402362127

In [31]:
from sklearn.metrics import confusion_matrix

y_test_predictions = modeling_pipeline.predict(X_test)

print(confusion_matrix(y_test, y_test_predictions))

[[65 38]
 [27 70]]


In [32]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_test_predictions))

              precision    recall  f1-score   support

          No       0.71      0.63      0.67       103
         Yes       0.65      0.72      0.68        97

    accuracy                           0.68       200
   macro avg       0.68      0.68      0.67       200
weighted avg       0.68      0.68      0.67       200



In [33]:
def generate_probs(X, model=modeling_pipeline):
    return model.predict_proba(X)[:, 1]
from sklearn.metrics import roc_auc_score

print ('training: ', roc_auc_score(y_train, generate_probs(X_train)))
print ('testing: ', roc_auc_score(y_test, generate_probs(X_test)))

training:  0.8305154664949904
testing:  0.7533780402362127


## Add grid search and train

In [34]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression

modeling_pipeline = Pipeline([('data_processing', processing_pipeline),
        #('pca', PCA(n_components=5)),
         ('logreg', LogisticRegression(penalty='l2', solver='liblinear'))
        ]
    )


from sklearn.model_selection import GridSearchCV

param_grid = [
  {
    'logreg__class_weight': [None, 'balanced'], 
    'logreg__C':[0.01, 0.1, 1, 10, 100],
    #'pca__n_components': [1,2,3,4,5,10,15]
  }
 ]

gcv_results = GridSearchCV(estimator=modeling_pipeline, param_grid=param_grid, scoring='accuracy')
gcv_results = gcv_results.fit(X_train, y_train)

In [35]:
gcv_results

GridSearchCV(estimator=Pipeline(steps=[('data_processing',
                                        ColumnTransformer(transformers=[('proc_numeric',
                                                                         Pipeline(steps=[('standardize_num',
                                                                                          StandardScaler())]),
                                                                         ['commuting_distance',
                                                                          'last_raise_pct',
                                                                          'total_years_working',
                                                                          'years_since_last_promotion',
                                                                          'years_with_current_supervisor']),
                                                                        ('create_dummies',
                                                                         Pipeline(steps=[('create_dummies_cats',
                                                                                          OneHotEn...
                                                                         ['frequency_of_travel',
                                                                          'department',
                                                                          'education',
                                                                          'satisfaction_with_environment',
                                                                          'gender',
                                                                          'seniority_level',
                                                                          'position',
                                                                          'satisfaction_with_job',
                                                                          'married_or_single',
                                                                          'last_performance_rating'])])),
                                       ('logreg',
                                        LogisticRegression(solver='liblinear'))]),
             param_grid=[{'logreg__C': [0.01, 0.1, 1, 10, 100],
                          'logreg__class_weight': [None, 'balanced']}],
             scoring='accuracy')

In [36]:
gcv_results.cv_results_.keys()

dict_keys(['mean_fit_time', 'std_fit_time', 'mean_score_time', 'std_score_time', 'param_logreg__C', 'param_logreg__class_weight', 'params', 'split0_test_score', 'split1_test_score', 'split2_test_score', 'split3_test_score', 'split4_test_score', 'mean_test_score', 'std_test_score', 'rank_test_score'])

In [37]:
gcv_results.best_score_

0.7212500000000001

In [38]:
gcv_results.best_params_

{'logreg__C': 100, 'logreg__class_weight': 'balanced'}

In [39]:
gcv_results.best_estimator_

Pipeline(steps=[('data_processing',
                 ColumnTransformer(transformers=[('proc_numeric',
                                                  Pipeline(steps=[('standardize_num',
                                                                   StandardScaler())]),
                                                  ['commuting_distance',
                                                   'last_raise_pct',
                                                   'total_years_working',
                                                   'years_since_last_promotion',
                                                   'years_with_current_supervisor']),
                                                 ('create_dummies',
                                                  Pipeline(steps=[('create_dummies_cats',
                                                                   OneHotEncoder(drop='first',
                                                                                 handle_unknown='ignore'))]),
                                                  ['frequency_of_travel',
                                                   'department', 'education',
                                                   'satisfaction_with_environment',
                                                   'gender', 'seniority_level',
                                                   'position',
                                                   'satisfaction_with_job',
                                                   'married_or_single',
                                                   'last_performance_rating'])])),
                ('logreg',
                 LogisticRegression(C=100, class_weight='balanced',
                                    solver='liblinear'))])

In [40]:
y_trainp = gcv_results.predict(X_train)
from sklearn.metrics import classification_report
print(classification_report(y_train, y_trainp))

              precision    recall  f1-score   support

          No       0.76      0.73      0.74       397
         Yes       0.74      0.77      0.76       403

    accuracy                           0.75       800
   macro avg       0.75      0.75      0.75       800
weighted avg       0.75      0.75      0.75       800



In [41]:
y_testp = gcv_results.predict(X_test)

from sklearn.metrics import classification_report
print(classification_report(y_test, y_testp))

              precision    recall  f1-score   support

          No       0.71      0.63      0.67       103
         Yes       0.65      0.72      0.68        97

    accuracy                           0.68       200
   macro avg       0.68      0.68      0.67       200
weighted avg       0.68      0.68      0.67       200



In [42]:
print(confusion_matrix(y_train, y_trainp))

[[289 108]
 [ 92 311]]


In [43]:
print(confusion_matrix(y_test, y_testp))

[[65 38]
 [27 70]]


## Which one is better? Explain?

**According to my findings, I can say that the base logistic regression model is same as Grid Search, in base logistic regression model, I got training accuracy of 76 and testing accuracy of 71 while in Grid Search, training accuracy is 76 and testing is 71 which clearly states that both of these models has the same result. But while Grid search, we got a clear output like which is the best score, parameter. From the output it is clear that training accuracy is higher than testing accuracy which may be due to various reasons such as hyperparameters are not chosen properly, overfitting (small datasets often lead to overfitting).** 